# Amazon Augmented AI (Amazon A2I) integration with Amazon Textract's Analyze Document [Example]

Visit https://github.com/aws-samples/amazon-a2i-sample-jupyter-notebooks for all A2I Sample Notebooks


## Introduction

Amazon Augmented AI (Amazon A2I) makes it easy to build the workflows required for human review of ML predictions. Amazon A2I brings human review to all developers, removing the undifferentiated heavy lifting associated with building human review systems or managing large numbers of human reviewers. 

Amazon A2I provides built-in human review workflows for common machine learning use cases, such as content moderation and text extraction from documents, which allows predictions from Amazon Rekognition and Amazon Textract to be reviewed easily. You can also create your own workflows for ML models built on Amazon SageMaker or any other tools. Using Amazon A2I, you can allow human reviewers to step in when a model is unable to make a high confidence prediction or to audit its predictions on an on-going basis. Learn more here: https://aws.amazon.com/augmented-ai/

In this tutorial, we will show how you can use Amazon A2I directly within your API calls to Textract's Analyze Document API. 

For more in depth instructions, visit https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-getting-started.html

To incorporate Amazon A2I into your human review workflows, you need three resources:

* A **worker task template** to create a worker UI. The worker UI displays your input data, such as documents or images, and instructions to workers. It also provides interactive tools that the worker uses to complete your tasks. For more information, see https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-instructions-overview.html

* A **human review workflow**, also referred to as a flow definition. You use the flow definition to configure your human workforce and provide information about how to accomplish the human review task. For built-in task types, you also use the flow definition to identify the conditions under which a review human loop is triggered. For example, with Amazon Textract can analyze text in a document using machine learning. You can use the flow definition to specify that a document will be sent to a human for content moderation review if Amazon Textracts's confidence score output is low for any or all pieces of text returned by Textract. You can create a flow definition in the Amazon Augmented AI console or with the Amazon A2I APIs. To learn more about both of these options, see https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html

* A **human loop** to start your human review workflow. When you use one of the built-in task types, the corresponding AWS service creates and starts a human loop on your behalf when the conditions specified in your flow definition are met or for each object if no conditions were specified. When a human loop is triggered, human review tasks are sent to the workers as specified in the flow definition.

When using a custom task type, you start a human loop using the Amazon Augmented AI Runtime API. When you call StartHumanLoop in your custom application, a task is sent to human reviewers.

### Install Latest SDK

In [ ]:
# First, let's get the latest installations of our dependencies
!pip install --upgrade pip
!pip install boto3 --upgrade
!pip install -U botocore

## Setup
We need to set up the following data:
* `region` - Region to call A2I
* `bucket` - A S3 bucket accessible by the given role
    * Used to store the sample images & output results
    * Must be within the same region A2I is called from
* `role` - The IAM role used as part of StartHumanLoop. By default, this notebook will use the execution role
* `workteam` - Group of people to send the work to

In [ ]:
# Region
REGION = 'us-east-1'

### Workteam or Workforce

A workforce is the group of workers that you have selected to label your dataset. You can choose either the Amazon Mechanical Turk workforce, a vendor-managed workforce, or you can create your own private workforce for human reviews. Whichever workforce type you choose, Amazon Augmented AI takes care of sending tasks to workers. 

When you use a private workforce, you also create work teams, a group of workers from your workforce that are assigned to Amazon Augmented AI human review tasks. You can have multiple work teams and can assign one or more work teams to each job.

To create your Workteam, visit the instructions here: https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-management.html

After you have created your workteam, replace YOUR_WORKTEAM_ARN below

In [ ]:
WORKTEAM_ARN= "arn:aws:sagemaker:us-east-1:077546553367:workteam/private-crowd/humanflow-team"

#### Setup Bucket and Paths

In [ ]:
import boto3
import botocore

BUCKET = 'textract-ocr-unicorn-gym-asean-demo'
OUTPUT_PATH = f's3://{BUCKET}/a2i-results'

In [ ]:
OUTPUT_PATH

### Permissions

The AWS IAM Role used to execute the notebook needs to have the following permissions:

* TextractFullAccess
* SagemakerFullAccess
* S3 Read Access to the bucket listed below
* AmazonSageMakerMechanicalTurkAccess (if using MechanicalTurk as your Workforce)

In [ ]:
from sagemaker import get_execution_role

# Setting Role to the default SageMaker Execution Role
ROLE = get_execution_role()
display(ROLE)

Visit: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-permissions-security.html to add the necessary permissions to your role

## Client Setup

Here we are going to setup the clients. 

In [ ]:
import boto3
import io
import json
import uuid
import botocore
import time
import botocore

# Amazon SageMaker client
sagemaker = boto3.client('sagemaker', REGION)

# Amazon Textract client
textract = boto3.client('textract', REGION)

# S3 client
s3 = boto3.client('s3', REGION)

# A2I Runtime client
a2i_runtime_client = boto3.client('sagemaker-a2i-runtime', REGION)

In [ ]:
import pprint

# Pretty print setup
pp = pprint.PrettyPrinter(indent=2)

# Function to pretty-print AWS SDK responses
def print_response(response):
    if 'ResponseMetadata' in response:
        del response['ResponseMetadata']
    pp.pprint(response)

## Create Control Plane Resources

### Create Human Task UI

Create a human task UI resource, giving a UI template in liquid html. This template will be rendered to the human workers whenever human loop is required.

We are providing a simple demo template that is compatible with A2I API input and response.

In [ ]:
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
  <img style="max-width: 35vw; max-height: 50vh" src="{{ task.input.taskObject | grant_read_access }}">
  <p><label>{{ task.input.date }}</label></p>
  <crowd-input name="date" label="date" required></crowd-input>
  <crowd-input name="tag1" label="Word/phrase 1"></crowd-input>
  <p><label>{{ task.input.vendor }}</label></p>
  <crowd-input name="vendor" label="vendor" required></crowd-input>
  <p><label>{{ task.input.total }}</label></p>
  <crowd-input name="total" label="total" required></crowd-input>

  <short-instructions>
    Your custom quick instructions and examples
  </short-instructions>

  <full-instructions>
    Your custom detailed instructions and more examples
  </full-instructions>
</crowd-form>
"""

def create_task_ui(task_ui_name):
    '''
    Creates a Human Task UI resource.

    Returns:
    struct: HumanTaskUiArn
    '''
    response = sagemaker.create_human_task_ui(
        HumanTaskUiName=task_ui_name,
        UiTemplate={'Content': template})
    return response

In [ ]:
# Task UI name - this value is unique per account and region. You can also provide your own value here.
taskUIName = 'ui-textract-demo'

# Create task UI
humanTaskUiResponse = create_task_ui(taskUIName)
humanTaskUiArn = humanTaskUiResponse['HumanTaskUiArn']
print(humanTaskUiArn)

### Creating the Flow Definition

In this section, we're going to create a flow definition definition. Flow Definitions allow us to specify:

* The conditions under which your human loop will be called.
* The workforce that your tasks will be sent to.
* The instructions that your workforce will receive. This is called a worker task template.
* The configuration of your worker tasks, including the number of workers that receive a task and time limits to complete tasks.
* Where your output data will be stored.

This demo is going to use the API, but you can optionally create this workflow definition in the console as well. 

For more details and instructions, see: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html.

#### Specify Human Loop Activation Conditions

Since we are using a built-in integration type for A2I, we can use Human Loop Activation Conditions to provide conditions that trigger a human loop.

Here we are specifying conditions for specific keys in our document. If Textract's confidence falls outside of the thresholds set here, the document will be sent to a human for review, with the specific keys that triggered the human loop prompted to the worker. 

In [ ]:
def create_flow_definition(flow_definition_name):
    '''
    Creates a Flow Definition resource

    Returns:
    struct: FlowDefinitionArn
    '''

    response = sagemaker.create_flow_definition(
            FlowDefinitionName= flow_definition_name,
            RoleArn= ROLE,
            HumanLoopConfig= {
                "WorkteamArn": WORKTEAM_ARN,
                "HumanTaskUiArn": humanTaskUiArn,
                "TaskCount": 1,
                "TaskDescription": "Document analysis sample task description",
                "TaskTitle": "Document analysis sample task"
            },
            OutputConfig={
                "S3OutputPath" : OUTPUT_PATH
            }
        )
    
    return response['FlowDefinitionArn']

#### Now we are ready to create our Flow Definition!

In [ ]:
# Flow definition name - this value is unique per account and region. You can also provide your own value here.
uniqueId = str(uuid.uuid4())
flowDefinitionName = f'fd-textract-{uniqueId}' 

flowDefinitionArn = create_flow_definition(flowDefinitionName)

In [ ]:
def describe_flow_definition(name):
    '''
    Describes Flow Definition

    Returns:
    struct: response from DescribeFlowDefinition API invocation
    '''
    return sagemaker.describe_flow_definition(
        FlowDefinitionName=name)

# Describe flow definition - status should be active
for x in range(60):
    describeFlowDefinitionResponse = describe_flow_definition(flowDefinitionName)
    print(describeFlowDefinitionResponse['FlowDefinitionStatus'])
    if (describeFlowDefinitionResponse['FlowDefinitionStatus'] == 'Active'):
        print("Flow Definition is active")
        break
    time.sleep(2)

### Test start human loop

Now that we have setup our Flow Definition, we can test a manual trigger of human loop

In [ ]:
    import uuid
    import boto3
    import json
    
    uniqueId = str(uuid.uuid4())
    human_loop_unique_id = uniqueId + '1'

    # A2I Runtime client
    a2i_runtime_client = boto3.client('sagemaker-a2i-runtime', 'us-east-1')
    
    s3_fname='s3://textract-ocr-unicorn-gym-asean-demo/data_extracted/X00016469671.jpg'
    
    # working key value set
    SAMPLE_BLOCK=[{'BlockType': 'KEY_VALUE_SET', 'Confidence': 51.06825256347656, 'Geometry': {'BoundingBox': {'Width': 0.16511861979961395, 'Height': 0.030581679195165634, 'Left': 0.04383595660328865, 'Top': 0.4065403342247009}, 'Polygon': [{'X': 0.04423863813281059, 'Y': 0.4065403342247009}, {'X': 0.2089545726776123, 'Y': 0.4105478823184967}, {'X': 0.20855188369750977, 'Y': 0.43712201714515686}, {'X': 0.04383595660328865, 'Y': 0.4331144690513611}]}, 'Id': '2d01a38a-4208-451d-ac51-6f6e428619c5', 'Relationships': [{'Type': 'VALUE', 'Ids': ['fa8dca94-4396-4dee-b95b-abc9cbd95783']}, {'Type': 'CHILD', 'Ids': ['c4616e22-3c21-4ab0-b5ae-70a91b80c951', '20506ce7-85bc-4631-b058-3d4d7f38d6cb']}], 'EntityTypes': ['KEY']}]
    
    
    inputContent = {
        "Blocks": SAMPLE_BLOCK,
        "initialValue": "hello",
        "taskObject": s3_fname,
        "date": "02/01/2019",
        "vendor": "OJC Marketing",
        "total" : "170"
    }
    
    a2i_response = a2i_runtime_client.start_human_loop(
        HumanLoopName=human_loop_unique_id,
        #FlowDefinitionArn='arn:aws:sagemaker:us-east-1:077546553367:flow-definition/test-custom-humanflow-v14',
        FlowDefinitionArn=flowDefinitionArn
        
        HumanLoopInput={
            'InputContent': json.dumps(inputContent)
        },
        DataAttributes={
            'ContentClassifiers': [
                'FreeOfPersonallyIdentifiableInformation'
            ]
        }
    )
    
    print(a2i_response)

## The End!